In [1]:
from detr.explainer import DetrExplainer
from pathlib import Path
from PIL import Image

import transformers as tr
import torch

In [2]:
model = tr.DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50", 
    ignore_mismatched_sizes=True)

processor = tr.DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\snovi\anaconda3\envs\pytorch_env\Lib\site-packages\transformers\models\detr

In [3]:
detr_explainer = DetrExplainer(
    model=model, 
    processor=processor, 
    id2label=model.config.id2label,
    no_object_id=len(model.config.id2label),
    device=device)

In [5]:
for img_path in Path(r'./resources/images/coco').glob('*.jpg'):
    img = Image.open(img_path)
    detr_explainer.explain(
        image=img,
        include_labels=['cat'],
        threshold=0.9,
        output_dir=Path(r'./explanations/') / img_path.stem)